In [ ]:
import os
%matplotlib inline
import nilearn
import numpy as np
import scipy
import math
from nilearn import datasets
from nilearn.input_data import NiftiMapsMasker
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
import nibabel as nib
import openpyxl
import matplotlib.pyplot as plt
import csv
import pandas as pd
from numpy import genfromtxt

In [ ]:
# save matrix in a vector, removing its diagonal
def reshape_no_diag(input_matrix):
    L = np.size(input_matrix, axis = 0)
    H_vec = np.reshape(input_matrix, (1, L*L))
    for i in reversed(xrange(L)):
        H_vec = np.delete(H_vec, i*L+i)
    return H_vec

In [ ]:
def hitting_time_calculation(AtlasPath, ts, L):
#     Inputs to hitting_time_calculation function:
#     AtlasPath = "path to nifti file including atlas image", e.g. atlas_filename = "/Volumes/MMP1_rois.nii"
#     ts = time_series
#     L = atlas size

    correlation_measure = ConnectivityMeasure(kind='partial correlation')
    atlas_filename = AtlasPath
    masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True,
                               memory='nilearn_cache', verbose=5) 
        
    correlation_matrix = correlation_measure.fit_transform([ts])[0]  
    correlation_matrix = abs(correlation_matrix)
    C_vec = np.reshape(correlation_matrix, (1, L*L))
    thrshld = np.percentile(C_vec, 50)

    np.fill_diagonal(correlation_matrix, 0)
    
    A_matrix = np.array(correlation_matrix)
    A_matrix = A_matrix > thrshld
    
    D_matrix = np.zeros((L,L))
    D_inv = np.zeros((L,L))
    D_sqrt = np.zeros((L,L))
    D_sqrt_inv = np.zeros((L,L))
    for i in range(L):
        D_matrix[i,i] = np.sum(A_matrix[i])
        D_inv[i,i] = 1./D_matrix[i,i]
        D_sqrt[i,i] = np.sqrt(D_matrix[i,i])
        D_sqrt_inv[i,i] = 1./D_sqrt[i,i]

    p_matrix = np.dot(D_inv, A_matrix)
    
    # Graph Laplacian
    eye_matrix = np.eye(L,L)
    eye_P = eye_matrix - p_matrix

    G_Lap = np.dot(D_sqrt,eye_P)
    G_Lap_n = np.dot(G_Lap, D_sqrt_inv)

    [eig_val, eig_vec] = np.linalg.eigh(G_Lap_n)

    H = np.zeros((L,L))
    d = np.sum(D_matrix)
    for i in range(L):
        for j in range(L):
            deg_i = D_matrix[i,i]
            deg_j = D_matrix[j,j]
            for k in range(L):
                if eig_val[k] != min(eig_val):
                    t_i = (eig_vec[i,k]*eig_vec[i,k])/deg_i
                    t_j = (eig_vec[j,k]*eig_vec[j,k])/deg_j
                    t_ij = (eig_vec[i,k]*eig_vec[j,k])/np.sqrt(deg_i*deg_j)
                    H[i,j] = H[i,j] + d*(1./(eig_val[k]))*(t_j-t_ij)

    H = np.transpose(H)
    return H

In [ ]:
def feature_extraction(H):
    H_vec = reshape_no_diag(H)
    H_mean = np.mean(H_vec)
    H_var = np.var(H_vec)
    H_skew = scipy.stats.skew(H_vec)
    H_kurtosis = scipy.stats.kurtosis(H_vec)
    H_Q10 = np.percentile(H_vec,10)
    H_Q90 = np.percentile(H_vec,90)
    H_medskew = H_Q90 - H_Q10
    H_median = np.median(H_vec)  
    H_kelly = H_Q90 + H_Q10 - 2*H_median
    return H_mean, H_var, H_medskew, H_kurtosis, H_skew, H_median, H_kelly 

In [ ]:
import os
import scipy.stats
import numpy as np
from numpy import genfromtxt
import astropy.table as table
import astropy.units as u
import sys
from astropy.table import Table, Column, MaskedColumn
from astropy.io import ascii 
import openpyxl
import csv

In [ ]:
dataset = {'subject_id': [], 'mean': [], 'median': [], 'var': [], 'skew': [], 'kelly': [], 'kurtosis': [], 'medskew': []}

for ts, subject_id, original_confound in zip(time_series,
                                             dataset.subject_id,
                                             dataset.motion):
    ### calculate hitting time 
#     AtlasPath = "path to nifti file including atlas image"
    H = hitting_time_calculation(AtlasPath, ts, L)
    
    ### calculate hitting time properties
    H_mean, H_var, H_medskew, H_kurtosis, H_skew, H_median, H_kelly = feature_extraction(H)
    
    ### writing the extracted features
    dataset['subject_id'].append(subject_id)
    dataset['mean'].append(H_mean)
    dataset['median'].append(H_median)
    dataset['var'].append(H_var)
    dataset['medskew'].append(H_medskew)
    dataset['kurtosis'].append(H_kurtosis)
    dataset['kelly'].append(H_kelly)
    dataset['skew'].append(H_skew)